Laplace equation with following boundary conditions: 

$$
\begin{equation}
  \begin{gathered}
p=0 \text{ at } x=0\\
\frac{\partial p}{\partial x} = 0 \text{ at } x = L_x\\
p = 0 \text{ at }y = 0 \\
p = \sin \left(  \frac{\frac{3}{2}\pi x}{L_x} \right) \text{ at } y = L_y
  \end{gathered}
\end{equation}
$$


In [1]:
import numpy
from matplotlib import pyplot
%matplotlib inline

In [2]:
# Set the font family and size to use for Matplotlib figures.
pyplot.rcParams['font.family'] = 'serif'
pyplot.rcParams['font.size'] = 16

In [3]:
#function of 3D plot
#function of 3D plot
def plot_3d(x, y, p, label='$z$', elev=30.0, azim=45.0):
    """
    Creates a Matplotlib figure with a 3D surface plot
    of the scalar field p.

    Parameters
    ----------
    x : numpy.ndarray
        Gridline locations in the x direction as a 1D array of floats.
    y : numpy.ndarray
        Gridline locations in the y direction as a 1D array of floats.
    p : numpy.ndarray
        Scalar field to plot as a 2D array of floats.
    label : string, optional
        Axis label to use in the third direction;
        default: 'z'.
    elev : float, optional
        Elevation angle in the z plane;
        default: 30.0.
    azim : float, optional
        Azimuth angle in the x,y plane;
        default: 45.0.
    """
    fig = pyplot.figure(figsize=(8.0, 6.0))
    ax = mplot3d.Axes3D(fig)
    fig.add_axes(ax)
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel(label)
    X, Y = numpy.meshgrid(x, y)
    ax.plot_surface(X, Y, p, cmap=cm.viridis)
    ax.set_xlim(x[0], x[-1])
    ax.set_ylim(y[0], y[-1])
    ax.view_init(elev=elev, azim=azim)

In [4]:
def laplace_solution(x, y, Lx, Ly):
    """
    Computes and returns the analytical solution of the Laplace equation
    on a given two-dimensional Cartesian grid.

    Parameters
    ----------
    x : numpy.ndarray
        The gridline locations in the x direction
        as a 1D array of floats.
    y : numpy.ndarray
        The gridline locations in the y direction
        as a 1D array of floats.
    Lx : float
        Length of the domain in the x direction.
    Ly : float
        Length of the domain in the y direction.

    Returns
    -------
    p : numpy.ndarray
        The analytical solution as a 2D array of floats.
    """
    X, Y = numpy.meshgrid(x, y) #meshgrid allows to plot on the whole 2D space
    p = (numpy.sinh(1.5 * numpy.pi * Y / Ly) /
         numpy.sinh(1.5 * numpy.pi * Ly / Lx) *
         numpy.sin(1.5 * numpy.pi * X / Lx))
    return p

In [5]:
def l2_norm(p, p_ref):
    """
    Computes and returns the relative L2-norm of the difference
    between a solution p and a reference solution p_ref.

    Parameters
    ----------
    p : numpy.ndarray
        The solution as an array of floats.
    p_ref : numpy.ndarray
        The reference solution as an array of floats.

    Returns
    -------
    diff : float
        The relative L2-norm of the difference.
    """
    l2_diff = (numpy.sqrt(numpy.sum((p - p_ref)**2)) /
               numpy.sqrt(numpy.sum(p_ref**2)))
    return l2_diff

In [6]:
#set parameters
nx = 128
ny = 128
Lx = 5.0
Ly = 5.0
dx = Lx / (nx - 1)
dy = Ly / (ny - 1)

x = numpy.linspace(0.0, Lx, num = nx)
y = numpy.linspace(0.0, Ly, num = ny)

#set initial conditions
p0 = numpy.zeros((ny, nx))
p0[-1, :] = numpy.sin((1.5*numpy.pi*x) / Lx)

In [7]:
def laplace_2d_jacobi_numpy(p0, maxiter = 20000, rtol = 1e-6):
    
    p = p0.copy()
    diff = rtol + 1.0
    ite = 0
    conv = []
    
    while diff>rtol and ite<maxiter:
        pn = p.copy()
        p[1:-1, 1:-1] = 0.25 * (pn[1:-1, :-2] + pn[1:-1, 2:] +
                                pn[:-2, 1:-1] + pn[2:, 1:-1])
        
        # Apply 2nd-order Neumann condition (zero-gradient)
        # at the right boundary.
        p[1:-1, -1] = 0.25 * (2.0 * pn[1:-1, -2] +
                              pn[2:, -1] + pn[:-2, -1])
        # Compute the relative L2-norm of the difference.
        diff = l2_norm(p, pn)
        ite += 1
    return p, ite, diff

In [8]:
import time

In [9]:
tic = time.time()
p, ites, diff = laplace_2d_jacobi_numpy(p0, maxiter = 20000, rtol = 1e-8)
print('Jacobi relaxation: {} iterations'.format(ites) +
     'to reach a relative difference of {}'.format(diff))
toc = time.time()
print(toc - tic)

Jacobi relaxation: 19993 iterationsto reach a relative difference of 9.998616841218672e-09
4.542097568511963


Conducted on 2021 Dell G15 5155, powered by 3.3 GHz AMD Ryzen 7 5800.

In [10]:
#Checking error with analytical solution
p_exact = laplace_solution(x, y, Lx, Ly)

l2_norm(p, p_exact)

6.173551335287356e-05

Gauss Siedel:

$$
\begin{equation}
p^{k+1}_{i,j} = \frac{1}{4} \left(p^{k+1}_{i,j-1} + p^k_{i,j+1} + p^{k+1}_{i-1,j} + p^k_{i+1,j} \right)
\end{equation}
$$

In [11]:
def laplace_2d_gauss_seidel1(p0, maxiter = 20000, rtol = 1e-6):
    
    p = p0.copy()
    diff = rtol + 1.0
    nx, ny = p0.shape
    ite = 0
    while diff>rtol and ite<maxiter:
        pn = p.copy()
        for j in range(1, ny-1):
            for i in range(1, nx-1):
                p[j, i] = 0.25*(p[j-1, i] + p[j+1, i] + p[j, i-1] + p[j, i+1])
                
        for j in range(1, ny-1):
            p[j, -1] = 0.25*(p[j-1, -1] + p[j+1, -1] + 2.0*p[j, -2])
            
        diff = l2_norm(p, pn)
        ite += 1
        
    return p, ite

In [ ]:
%%timeit
laplace_2d_gauss_seidel1(p0, maxiter = 20000, rtol = 1e-6)

In [ ]:
import numba
from numba import jit

In [ ]:
@jit(nopython=True)
def laplace_2d_jacobi(p0, maxiter=20000, rtol=1e-6):
    """
    
    Parameters
    ----------
    p0 : numpy.ndarray
        The initial solution as a 2D array of floats.
    maxiter : integer, optional
        Maximum number of iterations to perform;
        default: 20000.
    rtol : float, optional
        Relative tolerance for convergence;
        default: 1e-6.
    
    Returns
    -------
    p : numpy.ndarray
        The solution after relaxation as a 2D array of floats.
    ite : integer
        The number of iterations performed.
    conv : list
        The convergence history as a list of floats.
    """
    ny, nx = p0.shape
    p = p0.copy()
    conv = []  # convergence history
    diff = rtol + 1.0  # initial difference
    ite = 0  # iteration index
    while diff > rtol and ite < maxiter:
        pn = p.copy()
        # Update the solution at interior points.
        for j in range(1, ny - 1):
            for i in range(1, nx - 1):
                p[j, i] = 0.25 * (pn[j, i - 1] + pn[j, i + 1] +
                                  pn[j - 1, i] + pn[j + 1, i])
        # Apply 2nd-order Neumann condition (zero-gradient)
        # at the right boundary.
        for j in range(1, ny - 1):
            p[j, -1] = 0.25 * (2.0 * pn[j, -2] +
                               pn[j - 1, -1] + pn[j + 1, -1])
        # Compute the relative L2-norm of the difference.
        diff = numpy.sqrt(numpy.sum((p - pn)**2) / numpy.sum(pn**2))
        conv.append(diff)
        ite += 1
    return p, ite, conv

#### diff = l2_norm(p, pn) doesn't work with @jit numba, hence the whole computation is written

In [ ]:
# Compute the solution using Jacobi relaxation method.
p, ites, conv_jacobi = laplace_2d_jacobi(p0,
                                         maxiter=20000, rtol=1e-8)
print('Jacobi relaxation: {} iterations '.format(ites) +
      'to reach a relative difference of {}'.format(conv_jacobi[-1]))

In [ ]:
%%timeit
laplace_2d_jacobi(p0, maxiter=20000, rtol=1e-8)

The for loop based jacobi function with numba jit runs faster than numpy based jacobi function. Although the difference is not to huge. 

In [ ]:
@jit(nopython=True)
def laplace_2d_gauss_seidel2(p0, maxiter = 20000, rtol = 1e-6):
    
    p = p0.copy()
    diff = rtol + 1.0
    nx, ny = p0.shape
    ite = 0
    conv = []
    while diff>rtol and ite<maxiter:
        pn = p.copy()
        for j in range(1, ny-1):
            for i in range(1, nx-1):
                p[j, i] = 0.25*(p[j-1, i] + p[j+1, i] + p[j, i-1] + p[j, i+1])
                
        for j in range(1, ny-1):
            p[j, -1] = 0.25*(p[j-1, -1] + p[j+1, -1] + 2.0*p[j, -2])
            
        diff = numpy.sqrt(numpy.sum((p - pn)**2) / numpy.sum(pn**2))
        ite += 1
        conv.append(diff)
    return p, ite, conv

In [ ]:
# Compute the solution using Gauss-Seidel relaxation method.
tic = time.time()
p, ites, conv_gs = laplace_2d_gauss_seidel2(p0,
                                           maxiter=20000, rtol=1e-8)
toc = time.time()
print('Gauss-Seidel relaxation: {} iterations '.format(ites) +
      'to reach a relative difference of {}'.format(conv_gs[-1]))
print(toc - tic)

## Successive Over-Relaxation (SOR)

$$
\begin{equation}
p^{k+1}_{i,j} = (1 - \omega)p^k_{i,j} + \frac{\omega}{4} \left(p^{k+1}_{i,j-1} + p^k_{i,j+1} + p^{k+1}_{i-1,j} + p^k_{i+1,j} \right)
\end{equation}
$$

SOR iterations are only stable for $0 < \omega < 2$. Note that for $\omega = 1$, SOR reduces to the Gauss-Seidel method.

If $\omega < 1$, that is technically an "under-relaxation" and it will be slower than Gauss-Seidel.  

If $\omega > 1$, that's the over-relaxation and it should converge faster than Gauss-Seidel.  


In [ ]:
b = 2*x + y
print(numpy.shape(b))

In [ ]:
@jit(nopython=True)
def laplace_2d_sor(p0, b, omega, maxiter=20000, rtol=1e-6):
    
    nx, ny = p0.shape
    p = p0.copy()
    diff = rtol + 1.0
    ite = 0
    conv = []
    while diff>rtol and ite<maxiter:
        pn = p.copy()
        for j in range(1, ny-1):
            for i in range(1, nx-1):
                p[j, i] = (1 - omega)*p[j, i] + (omega/4)*(p[j-1, i] + p[j, i-1]+ p[j+1, i] + p[j, i+1] + b[j, i])
        #Applying Neumann bc (zero-gradient) at right boundary
        for j in range(1, ny-1):
            p[j, -1] = 0.25*(p[j-1, -1] + p[j+1, -1] + 2.0*p[j, -2])
                    
        # Compute the relative L2-norm of the difference.
        diff = numpy.sqrt(numpy.sum((p - pn)**2) / numpy.sum(pn**2))
        conv.append(diff)
        ite += 1
    
    return p, ite, conv

In [ ]:
# Compute the solution using SOR method.
omega = 1.0
tic = time.time()
p, ites, conv_sor = laplace_2d_sor(p0, b, omega, maxiter=20000, rtol=1e-8)
toc = time.time()
print('SOR (omega={}): {} iterations '.format(omega, ites) +
      'to reach a relative difference of {}'.format(conv_sor[-1]))
print(toc - tic)

In [ ]:
# Compute the solution using SOR method.
omega = 1.5
tic = time.time()
p, ites, conv_sor = laplace_2d_sor(p0, x, y, omega,
                                   maxiter=20000, rtol=1e-8)
toc = time.time()
print('SOR (omega={}): {} iterations '.format(omega, ites) +
      'to reach a relative difference of {}'.format(conv_sor[-1]))
print(toc -tic)

### Tuned SOR

For square domains, it turns out that the ideal factor $\omega$ can be computed as a function of the number of nodes in one direction, e.g., `nx`.

$$
\begin{equation}
\omega \approx \frac{2}{1+\frac{\pi}{nx}}
\end{equation}
$$

In [ ]:
# Compute the solution using tuned SOR method.
omega = 2.0 / (1.0 + numpy.pi / nx)
tic = time.time()
p, ites, conv_opt_sor = laplace_2d_sor(p0, omega,
                                       maxiter=20000, rtol=1e-8)
toc = time.time()
print('SOR (omega={:.4f}): {} iterations '.format(omega, ites) +
      'to reach a relative difference of {}'.format(conv_opt_sor[-1]))
print(toc - tic)

In [ ]:
l2_norm(p, p_exact)

In [ ]:
# Plot the convergence history for different methods.
pyplot.figure(figsize=(9.0, 4.0))
pyplot.xlabel('Iterations')
pyplot.ylabel('Relative $L_2$-norm\nof the difference')
pyplot.grid()
pyplot.semilogy(conv_jacobi, label='Jacobi')
pyplot.semilogy(conv_gs, label='Gauss-Seidel')
pyplot.semilogy(conv_sor, label='SOR')
pyplot.semilogy(conv_opt_sor, label='Optimized SOR')
pyplot.legend()
pyplot.xlim(0, 20000);